In [42]:
from datetime import datetime
from os.path import exists as file_exists
import pandas as pd
from pprint import pprint as pp
import re
import requests
from time import sleep
import xmltodict
import Credentials      # Get API keys, etc.

apikey = Credentials.prod_api
baseurl = 'https://api-na.hosted.exlibrisgroup.com'
item_query = '/almaws/v1/bibs/{mms_id}/holdings/{holding_id}/items/{item_pid}?apikey={apikey}'

exported_csv = "FullItemList.csv"
filled_csv = "FilledEnumChron.csv"

In [150]:
### Get MMS/Holding/Item IDs, Descriptions, Locations from spreadsheet ###
# Depending on how the file was exported, column names may or may have either spaces or underscores
df = pd.read_csv(exported_csv, converters={'Item_ID': str, 'Holdings_ID': str, 'MMS_ID': str, 'Item ID': str, 'Holdings ID': str, 'MMS ID': str})

In [151]:
# Remove spaces from column names
df.columns = [c.replace(' ', '_') for c in df.columns]
# Rename certain columns
df = df.rename(columns={'Permanent_Location': 'Location', 'Item_Policy': 'Policy', 'Material_Type': 'Material'})
# # Pare it down to just the necessary columns
# df = df[['MMS_ID', 'Holdings_ID', 'Item_ID', 'Description', 'Location']]
# Strip leading/trailing space from Description
df.Description = df.Description.str.strip()
# Collapse multiple spaces within the Description
df.Description.replace(' +', ' ', regex=True, inplace=True)

In [152]:
# Add columns for the Enum/Chron fields
EC_fields = ['Enum_A', 'Enum_B', 'Chron_I', 'Chron_J']
df[EC_fields] = None

##### Just so we have a sample record with a Chron field filled
df.loc[0, 'Chron_I'] = 'Whatever'

In [153]:
def fill_and_extract(regex, these_fields):
    exp = re.compile(regex)
    for i, f in enumerate(these_fields):
        df[f] = df['Description'].str.extract(exp, expand=True)[i]

#### So that's the function. Now just call it repeatedly for each regex that you come up with:

In [154]:
# Match descriptions with just volume & nothing else
fill_and_extract(r'^v\.(\d)+$', ['Enum_A'])

#### Then once all those replacements are done, pull filled-in records out to a new dataframe

In [155]:
# Create a dataframe to hold JUST records that get filled
filled = pd.DataFrame()
# Populate the new dataframe with any records that now have at least one Enum/Chron field filled
filled = df.dropna(subset=EC_fields, thresh=1)

# Purge records from the original dataframe if they are now in the new one
df = df.loc[~df['Item_ID'].isin(filled['Item_ID'])]

# Add a timestamp column to the `filled` dataframe
filled.insert(0, "Timestamp", datetime.now())

#### Do the spreadsheet files

In [100]:
# Export the 'filled' dataframe to a CSV
needs_header=not file_exists(filled_csv)    # Creating the file, so.
filled.to_csv(filled_csv, mode='a', index=False, header=needs_header)
# Replace Full CSV with what remains
df.to_csv(exported_csv, mode='w', index=False)

In [ ]:
records = len(filled)
c = 0
for index, row in filled.fillna('').iterrows():
    c += 1
    if (c % (records/100) < 1):
        print(int(100*c/records), '% complete', sep='', end='\r')
        sleep(5)
    print(c, str(row['Item_ID']), str(row['Holdings_ID']), str(row['MMS_ID']), sep="\t")
    r = requests.get(''.join([baseurl,item_query.format(mms_id=str(row['MMS_ID']), holding_id=str(row['Holdings_ID']), item_pid=str(row['Item_ID']), apikey=apikey)]))
    rdict = xmltodict.parse(r.text)

In [169]:
rdict['item']['item_data']['enumeration_a'] = str(row['Enum_A'])
rdict['item']['item_data']['enumeration_b'] = str(row['Enum_B'])
rdict['item']['item_data']['chronology_i'] = str(row['Chron_I'])
rdict['item']['item_data']['chronology_j'] = str(row['Chron_J'])
# Set an internal note, if there's one available
if (rdict['item']['item_data']['internal_note_1'] is None):
    rdict['item']['item_data']['internal_note_1'] = 'Enum/Chron derived from Description'
elif (rdict['item']['item_data']['internal_note_2'] is None):
    rdict['item']['item_data']['internal_note_2'] = 'Enum/Chron derived from Description'
elif (rdict['item']['item_data']['internal_note_3'] is None):
    rdict['item']['item_data']['internal_note_3'] = 'Enum/Chron derived from Description'
else:
    print()
    print("No internal note available for item MMS ID", str(row['MMS_ID']))

In [133]:
rxml = xmltodict.unparse(rdict)
r = requests.put(''.join([baseurl, item_query.format(mms_id=row['MMS_ID'], holding_id=row['Holdings_ID'], item_pid=row['Item_ID'], apikey=apikey)]), data=rxml.encode('utf-8'), headers={'Content-Type': 'application/xml'})
